# Develop a machine learning program to identify when an article might be fake news. 

    Run by the UTK Machine Learning Club.

* train.csv: A full training dataset with the following attributes:

* id: unique id for a news article
* title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable

    * 1: unreliable (FAKE)
    * 0: reliable   (REAL)
    
* test.csv: A testing training dataset with all the same attributes at train.csv without the label.

* submit.csv: A sample submission that you can

# Importing the dependancies

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import re 
    #re - regular expression, its very useful for searching the words in text or paragraph 
                    # within the document

from nltk.corpus import stopwords
    # corpus- body of the text, 
    # NLTK - Natural Langueage tool kit
    # stopwords- are the words that doesnt add much value to the paragraph or text in the context
                # words like- the, a , an , where, what, when, how,..

from nltk.stem.porter import PorterStemmer
    # stemmer- takes a word and removes the prefix and suffix of that word
                # it gives the root word for the particular word

from sklearn.feature_extraction.text import TfidfVectorizer
    # TfidfVectorizer - converting the text to feature vectorors, 
    # feature vect are nothing but numbers

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Akash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# printin the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Collection and Pre-Processing

In [ ]:
# Loading the Data Frame into pandas

In [10]:
news_dataset= pd.read_csv('train.csv')
news_dataset.shape

(20800, 5)

In [12]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
# Check the No. of missing values in the data set
print(news_dataset.shape)
news_dataset.isnull().sum()

(20800, 5)


id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [17]:
# Lets Drop the values.
 #if alot of the values are missing - we need to opt for imputation
 # but in this data- we can replace the missing values with null strings 
 # cuz we have a lot of data - we can replace the missing ones. else we would impute it

# replacing the null values with empty strings - ''

news_dataset = news_dataset.fillna('')

In [67]:
# Merging the author and news title 
# this is what we are going to use for predictions
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title'] 

In [68]:
news_dataset.columns

Index(['id', 'title', 'author', 'text', 'label', 'content'], dtype='object')

In [69]:
print(news_dataset["content"])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [70]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [71]:
# seperating the data & label

X = news_dataset.drop('label',axis=1)
y= news_dataset['label']

In [72]:
print(X)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

# Stemming 

    Stemming is the process of reducing a word to its Root word
    
    example - 
            actor, actress, acting, acted - Rootword = ACT
            
    We need to reduce the text as much as possble to make the model more efficeint

In [73]:
port_stem = PorterStemmer() #instantiating the variable

In [74]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
        #[^a-zA-Z]- removes evertything tht is not alphabets like numbers, punctuatons etc
        # we want to remove all the extra things except for alphabets and words from the content feature
        # ' '- is given so that we replace the non alphabets with a space.
        
    stemmed_content = stemmed_content.lower()
        # convert all the letters and words to lower case
        
    stemmed_content = stemmed_content.split()
        # convert all the words and letter to a LIST
        
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
        # we are taking each word and performin the stemming 
        # we are removing the stopwords(insignificant words)
        # we use a for loop- to pass the words in data frame. 
        # and choose only the words that are "not" stopwords
        # then perform port_stem algo on the choosen words

           
    stemmed_content = " ".join(stemmed_content)
        # after doing the port_stemmer opersation
        # we need to join the stemmed words
        
    return stemmed_content

In [75]:
# apply the def stemming on to the dataset

news_dataset['content'] = news_dataset['content'].apply(stemming)

In [76]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [77]:
# Seperating the data and label

X = news_dataset['content'].values
y = news_dataset['label'].values

In [78]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [81]:
print(y)

[1 0 1 ... 0 1 1]


In [82]:
y.shape 

(20800,)

In [83]:
X.shape

(20800,)

# We have all the data in words, computer cant understand the data in words.

    we need to convert the words into numbers using
    TfidfVectorizer- 
        Tf - Term Freq
             - basically count the no. of time a particular word is repaating in the document
             - the repeatation tells the model that this word is important
             - and it assigns a particular numerical value to that word
             
        idf - Inverse Document freq.
             - Sometimes a word that is repeated several times dosnt have a meaning to it
             - and it reduces its importance values

                


In [84]:
# Converting the Text data to numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X)
#here X is the ["content"] feature

X = vectorizer.transform(X)

In [85]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [86]:
# Now lets feed this data to the machine larning model. 
# this time th machine/model can understand the data as its is now in numerical

In [87]:
# Splitting the dataset to Training and Testing Data

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=.2, 
                                                    stratify=y,
                                                    random_state=2)
# we are splitting the data to train adn test data. 
# 80% - Training the data

# stratify=y- if you dont mention this stratify, 
#the real news and fake newss wont be segregated into equal proportion

# randomstate =2 , 

# Lets train our model 
       using LogesticRegression Model

In [88]:
model = LogisticRegression()

In [89]:
model.fit(X_train, y_train)
# we need to train the model by giving training data of both X and y


LogisticRegression()

In [90]:
#Now the model is trained.
#Now Lets Evaluate the model and find its accurace score

# Evaluation
    * accuracy score
    
    * lets do the prediction 1st on the training data ie - X_train & y_train

In [91]:
X_train_prediction = model.predict(X_train)
    # we want to train our model to make prediction on the X_train data
    # all the predictoin in the model will be stored in 
    # X_train_prediction

# now we need to find the accuracy score of predictons of the model 
# wrt X_train data with the Labels y_train data  
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [92]:
print("Accuracy score of the taining data: ", training_data_accuracy*100)

Accuracy score of the taining data:  98.65985576923076


# Dont be happy yet !!
    Cuz our model is trained on the Traing data - data that it already knows 
    now lets test the model with the testing data

In [93]:
# Lets find the accuracy for test data

In [94]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [95]:
print('The accuracy score for the TEST data is: ', test_data_accuracy)

The accuracy score for the TEST data is:  0.9790865384615385


# Now we are gonna Build A Prediction System

In [96]:
X_new = X_test[1]
prediction = model.predict(X_new)
print(prediction)

if (prediction[0] ==0):
    print("The news is REAL")
else:
    print("The News is FAKE")

[0]
The news is REAL


In [98]:
#Checking if the value is correct
print(y_test[1])

0
